In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd

# SBERT 모델 로드
embedder = SentenceTransformer("jhgan/ko-sbert-sts")

# 데이터 준비
review = pd.read_csv("../data/filtered_review_with_sentiment_scores.csv")
review['리뷰 텍스트'] = review['리뷰 텍스트'].fillna('').astype(str)
texts = review['리뷰 텍스트'].tolist()

# 배치 처리로 임베딩 생성
batch_size = 32
embeddings = []
with open('embeddings.npy', 'wb') as f:
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_embeddings = embedder.encode(batch_texts, convert_to_tensor=False)
        np.save(f, batch_embeddings)


In [ ]:
embeddings = []

with open('embeddings.npy', 'rb') as f:
    while True:
        try:
            batch_embeddings = np.load(f)
            embeddings.append(batch_embeddings)
        except EOFError:
            break

embeddings_np = np.vstack(embeddings)

: 

In [ ]:
# DBSCAN 클러스터링
dbscan = DBSCAN(eps=1.0, min_samples=5, metric='cosine')
labels = dbscan.fit_predict(embeddings_np)
# 결과 저장
review['cluster'] = labels
print(review[['리뷰 텍스트', 'cluster']].head())